# Benchmarking different quantum compilers


## Load benchmarks

In [35]:
import os
import subprocess
from pathlib import Path

benchmark_files = list(Path(".").rglob("*.qasm"))

## Prepare compilers

In [36]:
compilers = dict()

## QisKit

In [37]:
from qiskit.compiler import transpile

In [38]:
def optimize_qiskit(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    opt_qc = transpile(qc, optimization_level=3, approximation_degree=1.0,
                       basis_gates=['rz', 'rx', 'h', 't', 'tdag', 's', 'cx'])
    return opt_qc.qasm()


compilers['qiskit'] = optimize_qiskit

## PyZX

In [39]:
import pyzx as zx

In [40]:
def optimize_pyzx(qasm):
    c = zx.Circuit.from_qasm(qasm)
    c = c.to_graph()
    zx.simplify.full_reduce(c)
    c.normalize()
    c_opt = zx.extract_circuit(c.copy(), quiet=True, optimize_cnots=3).to_basic_gates()
    return c_opt.to_qasm()


compilers['pyzx'] = optimize_pyzx

## VOQC

To install voqc run
```shell
git submodule update --remote
cd pyvoqc
opam pin voqc https://github.com/inQWIRE/mlvoqc.git#mapping
./install.sh
```
Then copy pyvoqc/lib to your site packages pyvoqc folder

In [41]:
from qiskit import QuantumCircuit
from pyvoqc.qiskit.voqc_pass import voqc_pass_manager

In [42]:
def optimize_voqc(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    vpm = voqc_pass_manager(post_opts=["optimize"])
    qc_opt = vpm.run(qc)
    return qc_opt.qasm()


compilers['voqc'] = optimize_voqc

## $\mid$tket$\rangle$

In [43]:
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm_str
from pytket.passes import SequencePass, RemoveRedundancies, FullPeepholeOptimise, auto_rebase_pass
from pytket.predicates import CompilationUnit
from pytket import OpType

In [44]:
def optimize_tket(qasm):
    qc = circuit_from_qasm_str(qasm)
    gates = {OpType.Rz, OpType.Rx, OpType.H, OpType.T, OpType.Tdg, OpType.S, OpType.CX}
    rebase = auto_rebase_pass(gates)
    seqpass = SequencePass(
        [
            FullPeepholeOptimise(),
            RemoveRedundancies(),
            rebase,
        ])
    cu = CompilationUnit(qc)
    seqpass.apply(cu)
    qc_opt = cu.circuit
    return circuit_to_qasm_str(qc_opt)


compilers['tket'] = optimize_tket

## Staq

In [45]:
import pystaq


In [46]:
def optimize_staq(qasm):
    qc = pystaq.parse_str(qasm)
    pystaq.simplify(qc)
    pystaq.rotation_fold(qc)
    pystaq.simplify(qc)
    pystaq.cnot_resynth(qc)
    pystaq.simplify(qc)
    return str(qc)


compilers['staq'] = optimize_staq


# Run Benchmarks

## Gate counting

We count gates by finding them in the circuit representation.
Though, we assume a error corrected gate set, meaning we will synthesize any rotations that are not of form $c \times \frac{pi}{4}$ using Peter Sellinger's gridsynth (https://www.mathstat.dal.ca/~selinger/newsynth/)

In [47]:
import subprocess


def gridsynth(theta):
    return subprocess.getoutput(f"./gridsynth {theta} -e 10e-6")

In [48]:
from math import pi


def count_gates(qasm, gate_names):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in gate_names:
            count += 1
    return count


def approx(a, b):
    return abs(a - b) < 10e-4


def count_gates_synth(theta):
    synth = gridsynth(theta)
    return len(synth) - synth.count('W')


def count_t_synth(theta):
    synth = gridsynth(theta)
    return synth.count('T')


def count_rot_z_t(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += approx(pi / 4, abs(theta % (pi / 2)))
            else:
                count += count_t_synth(theta)

    return count


def count_t_gates(qasm):
    return count_gates(qasm, ['t', 'tdag']) + count_rot_z_t(qasm)


def count_cnot_gates(qasm):
    return count_gates(qasm, ['cx', 'cz'])


def count_total_gates(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += 1
            else:
                count += count_gates_synth(theta)
        else:
            count += 1
    return count


In [50]:
from qiskit.circuit.library import MCMT

# Test
testing = False
if testing:
    qc = QuantumCircuit(2)
    qc.rz(0.132312, 0)
    qc.t(0)
    qc.h(1)
    qc.cnot(0, 1)
    qc.t(0)
    qc.cnot(0, 1)
    qc.h(0)
    qc.t(0)
    qc.t(1)
    qc.s(1)
    qc.cnot(1, 0)
    qc.rx(pi / 4, 1)

    print("Original")
    print(qc.draw())

    qasm = qc.qasm()

    for compiler_name in compilers:
        print(compiler_name)
        qasm_opt = compilers[compiler_name](qasm)
        qc_opt = QuantumCircuit.from_qasm_str(qasm_opt)
        print(qc_opt.draw())
        print(count_t_gates(qasm_opt))


Original
     ┌─────────────┐┌───┐     ┌───┐     ┌───┐┌───┐┌───┐           
q_0: ┤ Rz(0.13231) ├┤ T ├──■──┤ T ├──■──┤ H ├┤ T ├┤ X ├───────────
     └────┬───┬────┘└───┘┌─┴─┐└───┘┌─┴─┐├───┤├───┤└─┬─┘┌─────────┐
q_1: ─────┤ H ├──────────┤ X ├─────┤ X ├┤ T ├┤ S ├──■──┤ Rx(π/4) ├
          └───┘          └───┘     └───┘└───┘└───┘     └─────────┘
qiskit
     ┌─────────────┐┌──────────┐┌──────────┐       ┌─────────┐   ┌──────────┐ »
q_0: ┤ Rz(-1.4385) ├┤ Rx(3π/4) ├┤ Rz(-π/2) ├──■────┤ Rx(π/2) ├───┤ Rz(-π/2) ├─»
     └┬────────────┤└──────────┘└──────────┘┌─┴─┐┌─┴─────────┴─┐┌┴──────────┴┐»
q_1: ─┤ Rx(2.2176) ├────────────────────────┤ X ├┤ Rz(0.79021) ├┤ Rx(1.6686) ├»
      └────────────┘                        └───┘└─────────────┘└────────────┘»
«                    
«q_0: ───────────────
«     ┌─────────────┐
«q_1: ┤ Rz(-1.4725) ├
«     └─────────────┘
147
pyzx
     ┌────────────┐┌───┐┌─────────┐┌───┐      ┌───┐                         
q_0: ┤ Rz(1.7031) ├┤ H ├┤ Rz(π/4) ├┤ H ├─■────┤ H ├──

In [64]:
import time

total_time = dict()
total_gates = dict()
total_cnot_gates = dict()
total_t_gates = dict()
time_taken = dict()
cnot_gate_count = dict()
gate_count = dict()
t_gate_count = dict()

base_key = 'base'

# Initalize
total_time[base_key] = 0
total_gates[base_key] = 0
total_cnot_gates[base_key] = 0
total_t_gates[base_key] = 0
for compiler_name_1 in compilers:
    for compiler_name_2 in compilers:
        key = compiler_name_1 + compiler_name_2
        time_taken[key] = dict()
        cnot_gate_count[key] = dict()
        t_gate_count[key] = dict()
        gate_count[key] = dict()
        total_time[key] = 0
        total_gates[key] = 0
        total_cnot_gates[key] = 0
        total_t_gates[key] = 0

# Benchmark individually
completed_benchmarks = []
for benchmark_file in benchmark_files:
    if benchmark_file == "temp_in.qasm" or benchmark_file == "tmp.qasm":  # Ignore compiler artifacts
        continue
    print(benchmark_file)
    benchmark = QuantumCircuit.from_qasm_file(
        benchmark_file).decompose().qasm()  # Remove custom gate declarations since tket and voqc don't play nicely with them
    assert not benchmark == ""
    total_gates[base_key] += count_total_gates(benchmark)
    total_cnot_gates[base_key] += count_cnot_gates(benchmark)
    total_t_gates[base_key] += count_t_gates(benchmark)
    for compiler_name_1 in compilers:
        for compiler_name_2 in compilers:
            key = compiler_name_1 + compiler_name_2
            print(f"{compiler_name_1} + {compiler_name_2}")
            start = time.perf_counter()
            opt_qasm = compilers[compiler_name_1](benchmark)
            if compiler_name_1 != compiler_name_2:
                opt_qasm = compilers[compiler_name_2](benchmark)
            stop = time.perf_counter()

            temp_time_taken = stop - start
            print(f"Completed in {temp_time_taken} sec")


            total_time[key] += temp_time_taken
            time_taken[key][str(benchmark_file)] = temp_time_taken

            temp_gate_count = count_total_gates(opt_qasm)
            total_gates[key] += temp_gate_count
            gate_count[key][str(benchmark_file)] = temp_gate_count

            temp_cnot_count = count_cnot_gates(opt_qasm)
            total_cnot_gates[key] += temp_cnot_count
            cnot_gate_count[key][str(benchmark_file)] = temp_cnot_count

            temp_t_gate_count = count_t_gates(opt_qasm)
            total_t_gates[key] += temp_t_gate_count
            t_gate_count[key][str(benchmark_file)] = temp_t_gate_count

    completed_benchmarks.append(str(benchmark_file))



Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
qiskit + qiskit
Completed in 6.638427859999865 sec
qiskit + pyzx
Completed in 22.849521261000064 sec
qiskit + voqc
Completed in 2.8962885240002834 sec
qiskit + tket
Completed in 5.061579454999901 sec
qiskit + staq
Completed in 1.5652589960000114 sec
pyzx + qiskit
Completed in 11.390085833000285 sec
pyzx + pyzx
Completed in 9.215364987000157 sec
pyzx + voqc
Completed in 9.354675720999694 sec
pyzx + tket
Completed in 11.32616009000003 sec
pyzx + staq
Completed in 9.128587698000047 sec
voqc + qiskit
Completed in 1.809215346000201 sec
voqc + pyzx
Completed in 9.714477853000062 sec
voqc + voqc
Completed in 0.6877074379999613 sec
voqc + tket
Completed in 2.863027572000192 sec
voqc + staq
Completed in 0.7538704559997313 sec
tket + qiskit
Completed in 3.4743699249997917 sec
tket + pyzx
Completed in 11.115520403000119 sec
tket + voqc
Completed in 2.8987932009999895 sec
tket + tket
Completed in 2.303143916999943 sec
tket + staq
Completed in 2.57

In [65]:
# Print results
print(time_taken)
print(gate_count)
print(cnot_gate_count)
print(t_gate_count)
print(total_time)
print(total_gates)
print(total_cnot_gates)
print(total_t_gates)

{'qiskitqiskit': {'Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm': 6.638427859999865, 'Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm': 0.5556467639999028, 'Nam-benchmarks/Arithmetic_and_Toffoli/gf2^4_mult.qasm': 0.7133608670001195}, 'qiskitpyzx': {'Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm': 22.849521261000064, 'Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm': 0.784451934000117, 'Nam-benchmarks/Arithmetic_and_Toffoli/gf2^4_mult.qasm': 1.263603331000013}, 'qiskitvoqc': {'Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm': 2.8962885240002834, 'Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm': 0.7681972299997142, 'Nam-benchmarks/Arithmetic_and_Toffoli/gf2^4_mult.qasm': 1.344587190999846}, 'qiskittket': {'Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm': 5.061579454999901, 'Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm': 0.8074449680002544, 'Nam-benchmarks/Arithmetic_and_Toffoli/gf2^4_mult.qasm': 1.5829061919998821}, 'qiskitstaq': 

In [66]:
import pickle


def get_new_file(base_name):
    i = 0
    while os.path.exists(f"{base_name}{i}.pkl"):
        i += 1
    return open(f"{base_name}{i}.pkl", 'wb')


pickle.dump(completed_benchmarks, get_new_file("completed_benchmarks"))
pickle.dump(total_time, get_new_file("total_time"))
pickle.dump(time_taken, get_new_file("time"))
pickle.dump(total_gates, get_new_file("total_gates"))
pickle.dump(gate_count, get_new_file("gates"))
pickle.dump(total_t_gates, get_new_file("total_t_gates"))
pickle.dump(t_gate_count, get_new_file("t_gates"))
pickle.dump(total_cnot_gates, get_new_file("total_cnot_gates"))
pickle.dump(cnot_gate_count, get_new_file("cnot_gates"))